## layers

In [14]:
from keras.layers import Conv2D, normalization, ReLU, Dense, Dropout, Flatten, BatchNormalization, AvgPool2D
from keras.utils import to_categorical
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras import backend as K

In [17]:
K.function?

In [10]:
gene = ImageDataGenerator()

In [11]:
g

In [7]:
Model.evaluate?

In [4]:
import keras.layers

## 创建callback

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

## 将卷积网络的输出结果的最后一层卷积，可视化

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import keras.backend as K

import matplotlib.pyplot as mp
import cv2 as cv
import numpy as np

# 加载以训练好的resnet模型，基于imagenet
model = ResNet50(weights='imagenet')

# 测试图片路径
img_path = r'D:\softfiles\workspace\git\mypython\ml\data\objects\training\airplane\0013.jpg'
# 加载图片为PIL对象
img = image.load_img(img_path, target_size=(224, 224))
# 把pil转成array
x = image.img_to_array(img)
# 增加batch维度
x = np.expand_dims(x, axis=0)
# 处理成模型输入格式
x = preprocess_input(x)


# 模型预测
preds = model.predict(x)
# 获取概率前三的预测结果
print('Predicted:', decode_predictions(preds, top=3)[0])


## 以下内容，将实现可视化预测主要区域。将最终的预测输出，反向梯度传向最后一个conv层，本model即为add_16层，
# 获取最大概率预测的下标
argmax = np.argmax(presd[0])
# 获取最后一层argmax的输出结果
air_plane = model.output[:, argmax]
# 获取最后一个卷积层
last_conv_layer = model.get_layer('add_16')

# 将输出结果梯度传向最后的卷积层,去掉第一维[7, 7, 2048]
grads = K.gradients(air_plane, last_conv_layer.output)[0]
# 求梯度平均
pooled_grads = K.mean(grads, axis=(0, 1, 2))

# 构建新的输入，输出结构
iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])

# 将预测图片放入到新的结构中
pooled_grads_value, conv_layer_output_value = iterate([x])

for i in range(2048):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
    
heatmap = np.mean(conv_layer_output_value, axis=-1)

# 归一化处理
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)

mp.matshow(heatmap)
mp.show()

img = cv.imread(img_path)
heatmap = cv.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv.applyColorMap(heatmap, cv.COLORMAP_JET)
superimposed_img = heatmap * 0.4 + img
cv.imwrite('D://test.jpg', superimposed_img)